In [1]:
import ccxt
from datetime import datetime, timedelta, timezone
import sys
import pandas as pd
import time
import numpy as np
import math
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score
from sklearn.model_selection import cross_val_score
import ta
from scipy.signal import argrelextrema
import mplfinance as mpf
import matplotlib.pyplot as plt

In [2]:
ori_df = pd.read_csv('SOL_15_minute.csv', parse_dates=['timestamp'])
df = ori_df.copy()

In [3]:
df

,timestamp,open,high,low,close,volume,time_diff
0,2024-01-01 00:00:00,101.74,102.79,101.56,102.36,105526.00,NaN
1,2024-01-01 00:15:00,102.37,102.55,101.54,102.08,102729.00,0 days 00:15:00
2,2024-01-01 00:30:00,102.05,102.57,101.63,102.19,99849.00,0 days 00:15:00
3,2024-01-01 00:45:00,102.17,102.27,101.95,101.99,18505.00,0 days 00:15:00
4,2024-01-01 01:00:00,101.98,102.59,101.56,102.43,49602.00,0 days 00:15:00
...,...,...,...,...,...,...,...
17190,2024-06-28 01:30:00,149.34,150.92,149.27,149.67,173743.89,0 days 00:15:00
17191,2024-06-28 01:45:00,149.67,149.69,148.63,148.95,161093.73,0 days 00:15:00
17192,2024-06-28 02:00:00,148.96,149.36,148.00,148.08,87390.18,0 days 00:15:00
17193,2024-06-28 02:15:00,148.08,148.68,147.22,147.27,107710.84,0 days 00:15:00


In [4]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)


df['EMA10'] = df['close'].ewm(span=10, adjust=False).mean()
df['EMA40'] = df['close'].ewm(span=40, adjust=False).mean()
df['EMA100'] = df['close'].ewm(span=100, adjust=False).mean()


df['uptrend_start'] = (df['EMA10'] > df['EMA40']) & (df['EMA40'] > df['EMA100']) & (df['EMA10'].shift(1) <= df['EMA40'].shift(1))
df['downtrend_start'] = (df['EMA10'] < df['EMA40']) & (df['EMA40'] < df['EMA100']) & (df['EMA10'].shift(1) >= df['EMA40'].shift(1))
df['uptrend_end'] = (df['EMA10'] < df['EMA40']) | (df['EMA40'] < df['EMA100'])
df['downtrend_end'] = (df['EMA10'] > df['EMA40']) | (df['EMA40'] > df['EMA100'])

In [5]:
df[(df.index > '2024-06-07 12:30') & (df.index <= '2024-06-09 06:30') & ((df['downtrend_end'] == True) | (df['downtrend_start'] == True))]


,open,high,low,close,volume,time_diff,EMA10,EMA40,EMA100,uptrend_start,downtrend_start,uptrend_end,downtrend_end
timestamp,,,,,,,,,,,,,
2024-06-07 12:45:00,170.67,171.36,170.26,171.17,133502.02,0 days 00:15:00,171.620628,171.565030,171.600561,False,False,True,True
2024-06-07 13:00:00,171.17,171.27,170.64,171.04,77111.34,0 days 00:15:00,171.515059,171.539419,171.589461,False,True,True,False
2024-06-09 06:30:00,159.26,159.28,158.77,159.12,17152.61,0 days 00:15:00,158.656336,158.593156,160.036868,False,False,True,True


In [6]:
# df[(df.index > '2024-06-09 06:30') & (df.index <= '2024-06-09 18:30')]

In [7]:
trend_data = []

# Iterate through the DataFrame to find tr end start and end pairs
for i in range(1, len(df)):
    if df['uptrend_start'].iloc[i]:
        start_time = df.index[i]
        start_close = df['close'].iloc[i]
        trend = 'uptrend'
        for j in range(i + 1, len(df)):
            if df['uptrend_end'].iloc[j]:
                end_time = df.index[j]
                end_close = df['close'].iloc[j]
                if (end_close - start_close) / start_close >= 0.015:  # Check if the change is at least 1.5%
                    trend_data.append([start_time, trend, start_time, end_time, None, None, None, None, None, None])
                break
    elif df['downtrend_start'].iloc[i]:
        start_time = df.index[i]
        start_close = df['close'].iloc[i]
        trend = 'downtrend'
        for j in range(i + 1, len(df)):
            if df['downtrend_end'].iloc[j]:
                end_time = df.index[j]
                end_close = df['close'].iloc[j]
                if (start_close - end_close) / start_close >= 0.015:  # Check if the change is at least 1.5%
                    trend_data.append([start_time, trend, start_time, end_time, None, None, None, None, None, None])
                break

# Create the initial trend DataFrame


In [8]:
df.index[0]

Timestamp('2024-01-01 00:00:00')

In [9]:
trend_df = pd.DataFrame(trend_data, columns=['timestamp', 'trend', 'trend_start', 'trend_end', 'conditions_met_time', 'next_bar_time', 'next_bar_open', 'high', 'low', 'profit'])

# Function to check the conditions with a percent parameter
def check_conditions(df, start_idx, end_idx, percent):
    conditions_met = [False, False, False]
    for i in range(start_idx, end_idx):
        
        
        if df['EMA40'].iloc[i] * (1 - percent) < df['EMA10'].iloc[i] and df['EMA40'].iloc[i] * (1 - percent) < df['EMA100'].iloc[i]:
            conditions_met[0] = True
        elif df['EMA10'].iloc[i] > df['EMA40'].iloc[i] > df['EMA100'].iloc[i] :
            conditions_met[1] = True
        elif df['EMA40'].iloc[i] * (1 + percent) > df['EMA10'].iloc[i] and df['EMA40'].iloc[i] * (1 + percent) > df['EMA100'].iloc[i]:
            conditions_met[2] = True
            
#         if all(conditions_met):
        if sum(conditions_met) >= 2:
            return df.index[i]
    return None

# Iterate through the trend DataFrame to update conditions_met_time and check next 5 sticks
percent = 0.01  
for idx in trend_df.index:
    end_idx = df.index.get_loc(trend_df.at[idx, 'trend_end'])
    conditions_met_time = check_conditions(df, end_idx, min(end_idx + 100, len(df)), percent)
    if conditions_met_time:
        trend_df.at[idx, 'conditions_met_time'] = conditions_met_time
        conditions_met_idx = df.index.get_loc(conditions_met_time)
        
        # Check next 5 sticks for downtrend and uptrend conditions
        if trend_df.at[idx, 'trend'] == 'downtrend':
            for k in range(conditions_met_idx + 1, min(conditions_met_idx + 6, len(df) - 1)):
                if df['close'].iloc[k] > df['open'].iloc[k] and df['close'].iloc[k + 1] > df['open'].iloc[k + 1]:  # Two consecutive up bars
                    trend_df.at[idx, 'next_bar_time'] = df.index[k + 2]
                    trend_df.at[idx, 'next_bar_open'] = df['open'].iloc[k + 2]
                    break
        elif trend_df.at[idx, 'trend'] == 'uptrend':
            for k in range(conditions_met_idx + 1, min(conditions_met_idx + 6, len(df) - 1)):
                if df['close'].iloc[k] < df['open'].iloc[k] and df['close'].iloc[k + 1] < df['open'].iloc[k + 1]:  # Two consecutive down bars
                    trend_df.at[idx, 'next_bar_time'] = df.index[k + 2]
                    trend_df.at[idx, 'next_bar_open'] = df['open'].iloc[k + 2]
                    break

# Calculate high and low between trend_end and conditions_met_time and profit/loss
for idx in trend_df.index:
    if pd.notnull(trend_df.at[idx, 'conditions_met_time']):
        end_time = trend_df.at[idx, 'trend_end']
        conditions_met_time = trend_df.at[idx, 'conditions_met_time']
        interval_df = df.loc[end_time:conditions_met_time]
        high = interval_df['high'].max()
        low = interval_df['low'].min()
        entry_price = trend_df.at[idx, 'next_bar_open']
        
        # Ensure entry_price is not None before performing calculations
        if entry_price is not None:
            if trend_df.at[idx, 'trend'] == 'downtrend':
                stop_loss = high
                target = entry_price - (high - entry_price)  # 1:1 risk-to-reward ratio
                profit = target - entry_price
            else:  # uptrend
                stop_loss = low
                target = entry_price + (entry_price - low)  # 1:1 risk-to-reward ratio
                profit = target - entry_price
            
            trend_df.at[idx, 'high'] = high
            trend_df.at[idx, 'low'] = low
            trend_df.at[idx, 'profit'] = profit
        else:
            trend_df.at[idx, 'high'] = None
            trend_df.at[idx, 'low'] = None
            trend_df.at[idx, 'profit'] = None

In [13]:
trend_df

,timestamp,trend,trend_start,trend_end,conditions_met_time,next_bar_time,next_bar_open,high,low,profit
0,2024-01-01 09:30:00,uptrend,2024-01-01 09:30:00,2024-01-02 14:30:00,2024-01-03 00:45:00,None,None,None,None,None
1,2024-01-03 10:45:00,downtrend,2024-01-03 10:45:00,2024-01-04 02:45:00,2024-01-04 20:30:00,2024-01-04 22:15:00,106.33,108.31,96.61,-1.98
2,2024-01-06 01:15:00,downtrend,2024-01-06 01:15:00,2024-01-06 15:15:00,None,None,None,None,None,None
3,2024-01-07 20:45:00,downtrend,2024-01-07 20:45:00,2024-01-08 08:15:00,2024-01-08 18:15:00,2024-01-08 19:30:00,99.21,98.02,88.61,1.19
4,2024-01-23 06:30:00,downtrend,2024-01-23 06:30:00,2024-01-23 16:15:00,2024-01-24 09:45:00,2024-01-24 10:30:00,87.43,87.39,80.36,0.04
5,2024-01-26 08:15:00,uptrend,2024-01-26 08:15:00,2024-01-27 05:30:00,2024-01-27 06:30:00,None,None,None,None,None
6,2024-01-27 12:15:00,uptrend,2024-01-27 12:15:00,2024-01-28 17:30:00,2024-01-28 18:45:00,2024-01-28 20:00:00,94.53,96.51,95.13,-0.6
7,2024-01-29 15:45:00,uptrend,2024-01-29 15:45:00,2024-01-30 22:15:00,2024-01-30 23:15:00,2024-01-31 00:30:00,100.38,104.44,102.3,-1.92
8,2024-01-31 20:15:00,downtrend,2024-01-31 20:15:00,2024-02-01 10:15:00,2024-02-02 09:00:00,2024-02-02 10:45:00,101.86,101.56,94.71,0.3
9,2024-02-08 20:45:00,uptrend,2024-02-08 20:45:00,2024-02-09 16:45:00,2024-02-09 23:15:00,2024-02-10 00:00:00,107.0,109.31,104.36,2.64


In [11]:
trend_df.profit.sum()

13.44999999999996

In [12]:
#         if sum(conditions_met) >= 2:
#             return df.index[i]

In [19]:
139.01-138.17

0.8400000000000034

In [ ]:
conditions_met_time next_bar_time next_bar_open high low profit